# PROSJEKT INF161
## Movie recommendation project - clean up
`@author` Kristin Loka Øydna

### Background
In this project I will build a movie recommender system to recommend the 10 best movies to a spesific user that they have not seen. 

The raw data files contain 900 188 anonymous ratings of 3900 movies done by 6040 users. The users have listed gender, age-group and job. 

Age is divided into the following groups:

*  1:  "Under 18"
* 18:  "18-24"
* 25:  "25-34"
* 35:  "35-44"
* 45:  "45-49"
* 50:  "50-55"
* 56:  "56+"

Job are divided into:

*  0:  "annet" eller ikke spesifisert
*  1:  "akademisk/pedagog"
*  2:  "kunstner"
*  3:  "administrativt"
*  4:  "student"
*  5:  "kundeservice"
*  6:  "helse"
*  7:  "ledende"
*  8:  "bonde"
*  9:  "hjemmeværende"
* 10:  "elev"
* 11:  "advokat"
* 12:  "programmerer"
* 13:  "pensjonert"
* 14:  "salg/markedsføring"
* 15:  "forsker"
* 16:  "selvstendig næringsdrivende"
* 17:  "tekniker/ingeniør"
* 18:  "håndverker"
* 19:  "arbeidsledig"
* 20:  "forfatter"

Gender is male (M) or female (F).

Timestamp in the rating dataset is represented in seconds, starting from 01.01.1970.


The project includes of the following files:
* cleanup.ipynb 
 - Cleaned up raw data
 - Generates a folder `clean_data` consisting of the csv files: `bruker.csv`, `film.csv` and `rangering.csv` 

* model.ipynb
 - Visualization and statistics
 - Generates a machine learning model that is saved to disk
 
* predict.ipynb
 - Loads a machine learning model and user data, and prints out predictions
 
* predictions.csv
 - Matrix with rating predictions for each user and each movie
 
* A zip file containing the file app.py and all neccesary files
 - Runs a local movie recommendation website

I did not get a matrix with rating predictions for each user and each movie, and therefor not a working website.

In [1]:
import pandas as pd
import numpy as np
import json
import os
import csv
import datetime as dt
from numpy import nan
from math import ceil
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

### Cleaning up raw data

I created a new folder, `cleaned_data`, where all the cleaned up dataframes for ratings, movies and users are saved as csv files after they are cleaned. The raw data we got handed out consists of the files `bruker.json`, `film.xlsx` and `rangering.dat`. All three dataframes had to be cleand for further use. 

In the user dataframe we had to split the data into separate columns before cleaning it. There are errors and missing values in the columns `Kjonn`, `Alder`, `Jobb` and `Tidsstempel` that have to be delt with. Jobs are set between 0 and 20. All missing jobs, and jobs that are not in this intervall I have set to 0 since this is categorised as "other", or not spesified. Postcodes consist of 5 digits, however, at least one user have a postcode consisting of more digits. This is corrected, and missing postcodes are replaced by "Unknown". For gender and age I chose to replace the missing values by the most frequent value. To get a more realistic picture I split the dataset and used mode on the training set. 

In the movie dataset all genres to a movie was in one column. I separated all the genres into different columns, and if a movie is in that genre it is marked with 1, if not 0. In the originale dataset we were suppose to compare it with there was 18 genres, here we hav 20. I found out that children's movies was placed in two genres spelled different. I had to merge these columns into one. There was also a "Ukjennt" column with movies with unknown genre. I dropped this column, and rows with the movies since they did not belong to a genre and there was only 3 movies, and the rest of this report is based mostly on genres. 

In the ratings dataset I split the data into columns: `BrukerID`, `FilmID`, `Rangering` and `Tidsstempel`. I have cleaned both rantings and timestamp before splitting the data to make sure they are changed for the entire dataset. In timestamp I filled in the missing values based on the next timestamp since the timestamp is most likely following a pattern. We want the ratings to be on the form 1-5, but before 01.08.2000 they are on the form 1-10. I changed the old ratings by dividing them by 2 and using the ceil function to make sure the ratings are placed in the right interval. 

In [2]:
# Create new folder
path = r'/Users/kristinlokaoydna/UiB - Bioinformatikk/H20/INF161/Prosjekt/cleaned_data'
if not os.path.exists(path):
    os.makedirs(path)

# Read bruker.json
os.chdir('/Users/kristinlokaoydna/UiB - Bioinformatikk/H20/INF161/Prosjekt/')
with open('bruker.json') as users:
    users = json.load(users)
    
bruker = pd.DataFrame.from_dict(users, orient='index').T.set_index('index')


bruker[['BrukerID','Kjonn', 'Alder', 'Jobb', 'Postkode']] = pd.DataFrame(bruker.data.values.tolist(), index= bruker.index)
bruker_df = pd.DataFrame(bruker['data'].values.tolist(), columns=['BrukerID','Kjonn', 'Alder', 'Jobb', 'Postkode'])


# Read film.xlsx
film_df = pd.read_excel(r'/Users/kristinlokaoydna/UiB - Bioinformatikk/H20/INF161/Prosjekt/film.xlsx', sheet_name = 'film')
film_df = film_df.drop(['Unnamed: 0'], axis = 1)


# Generate new column for every genre
genre_list = ""
for index,row in film_df.iterrows():
        genre_list += row.Sjanger + "|"
genre_list_split = genre_list.split('|')
new_list = list(set(genre_list_split))
new_list.remove('')

film_kopi = film_df.copy()
for genre in new_list:
    film_kopi[genre] = film_kopi.apply(lambda x: int(genre in x.Sjanger), axis=1)
film_df = film_kopi.drop(['Sjanger'], axis = 1)


# Create ONE "Children's" genre
film_df["Child"] = film_df["Children"] + film_df["Children's"]
film_df.loc[film_df["Child"] == 2, "Child"] = 1
film_df = film_df.drop(["Children's", "Children"], axis = 1)
film_df = film_df.rename(columns={"Child": "Children's"})

sort_genre = film_df.iloc[:,2:].sort_index(axis=1)
film_df = film_df.iloc[:, :2].join(sort_genre)

# Checking if there is any films in the genre "Ukjennt" and remove them
print("Amount of movies with unknown genres:",film_df['Ukjennt'].sum())

remove = []
for index, row in film_df.iterrows():
    if row['Ukjennt']==1:
        remove.append(row.name)
film_df = film_df.drop(remove, axis = 0)
film_df = film_df.drop(['Ukjennt'], axis = 1)


# Read rangering.dat
with open("/Users/kristinlokaoydna/UiB - Bioinformatikk/H20/INF161/Prosjekt/rangering.dat") as file:
    with open("new_rangering.csv", "w") as file1:
        for line in file:
            file1.write(line)
rang = pd.read_csv('new_rangering.csv')

# Splitter data inn i kolonner og gir dem navn
rang = rang.rename(columns={"0::616::4::959441640.0": "data"})
rang_df = rang.data.str.split(r'::', expand=True)
rang_df = rang_df.rename(columns={0: 'BrukerID', 1:'FilmID', 2:'Rangering', 3:'Tidsstempel'})


Amount of movies with unknown genres: 3


In [3]:
# Amount of missing values that have to be fixed
print("Missing values in the three dataframes")
print(rang_df.isnull().sum())
print(bruker_df.isnull().sum())
print(film_df.isnull().sum())

Missing values in the three dataframes
BrukerID       0
FilmID         0
Rangering      0
Tidsstempel    0
dtype: int64
BrukerID      0
Kjonn       303
Alder       994
Jobb        593
Postkode    451
dtype: int64
FilmID         0
Tittel         0
Action         0
Adventure      0
Animation      0
Children's     0
Comedy         0
Crime          0
Documentary    0
Drama          0
Fantasy        0
Film-Noir      0
Horror         0
Musical        0
Mystery        0
Romance        0
Sci-Fi         0
Thriller       0
War            0
Western        0
dtype: int64


In [4]:
# Fixing job
for i in bruker_df["Jobb"]:
    if i > 20:
        bruker_df["Jobb"].replace(i, 0, inplace=True)

bruker_df["Jobb"].replace(nan, 0, inplace=True)


#Change postcode to 5 digits
bruker_df["Postkode"].replace(nan, "Unknown", inplace=True)
print("Any postcode with more than 5 digits:",(bruker_df["Postkode"].str.len()>5).any())

pcode = []
for code in bruker_df["Postkode"]:
    if code == "Unknown":
        pcode.append(code)
    elif len(code) > 5:
        pcode.append(code[:5])    
    else:
        pcode.append(code)

bruker_df = bruker_df.drop(["Postkode"], axis = 1)

bruker_df["Postkode"] = pcode

Any postcode with more than 5 digits: True


In [5]:
# Filling missing values in timestamp, and changing the rating system so all are on the form 1-5
end = dt.datetime(2000,7,31,23,59,59)
start = dt.datetime(1970,1,1,0,0,0)

time = (end-start).total_seconds()
            
rang_df.replace("", np.NaN, inplace=True)

rang_df = rang_df.astype({"BrukerID": "int64", "FilmID": "int64", "Rangering": "int64", "Tidsstempel": "float"})
rang_df["Tidsstempel"] = rang_df["Tidsstempel"].fillna(method = 'ffill')
        
for index in rang_df.index:
    if rang_df.at[index, "Tidsstempel"] < time: 
        change = np.ceil(rang_df.at[index, "Rangering"] / 2)
        rang_df.at[index, "Rangering"] = change

### Splitting the dataset
Before I merged the rating, movie and user dataframes I chose to split the rating dataset first and then merge this with the other dataset to create a train, validation and test set. This ensures that movies that are not rated is not included in the merged dataset. The dataset is split into train (70%), validation (15%) and test (15%).

In [6]:
# Split rating dataset
X_r = rang_df.drop(columns="Rangering")
y_r = rang_df["Rangering"]

X_r_train, X_r_valtest, y_r_train, y_r_valtest = train_test_split(X_r, y_r, test_size=0.3, random_state=42)
X_r_val, X_r_test, y_r_val, y_r_test = train_test_split(X_r_valtest, y_r_valtest, test_size=0.5, random_state=42)


In [7]:
#Merging the dataset to train dataset
train_r = X_r_train.copy()
train_r['Rangering'] = y_r_train
train = pd.merge(train_r,film_df, on = "FilmID")
train = pd.merge(train,bruker_df, on = "BrukerID")

#Merging the dataset to val dataset
val_r = X_r_val.copy()
val_r['Rangering'] = y_r_val
val = pd.merge(val_r,film_df, on = "FilmID")
val = pd.merge(val,bruker_df, on = "BrukerID")

#Merging the dataset to test dataset
test_r = X_r_test.copy()
test_r['Rangering'] = y_r_test
test = pd.merge(test_r,film_df, on = "FilmID")
test = pd.merge(test,bruker_df, on = "BrukerID")

# Merging the train, val and test into one datasets
merged = pd.concat([train,val, test] )

In [8]:
# Replacing missing values in gender and age based on most frequent value in train set

merged["Kjonn"].fillna(train["Kjonn"].mode()[0], inplace=True)
merged["Alder"].fillna(train["Alder"].mode()[0], inplace=True)

print("Missing valus in merged dataset:")
merged.isnull().sum()

Missing valus in merged dataset:


BrukerID       0
FilmID         0
Tidsstempel    0
Rangering      0
Tittel         0
Action         0
Adventure      0
Animation      0
Children's     0
Comedy         0
Crime          0
Documentary    0
Drama          0
Fantasy        0
Film-Noir      0
Horror         0
Musical        0
Mystery        0
Romance        0
Sci-Fi         0
Thriller       0
War            0
Western        0
Kjonn          0
Alder          0
Jobb           0
Postkode       0
dtype: int64

In [9]:
merged

,BrukerID,FilmID,Tidsstempel,Rangering,Tittel,Action,Adventure,Animation,Children's,Comedy,...,Mystery,Romance,Sci-Fi,Thriller,War,Western,Kjonn,Alder,Jobb,Postkode
0,4722,630,1.028508e+09,3,"Perfect Murder, A (1998)",0,0,0,0,0,...,1,0,0,1,0,0,M,25.0,0.0,92346
1,4722,3138,9.746983e+08,1,"Story of Us, The (1999)",0,0,0,0,1,...,0,0,0,0,0,0,M,25.0,0.0,92346
2,4722,3240,1.028508e+09,4,"Shawshank Redemption, The (1994)",0,0,0,0,0,...,0,0,0,0,0,0,M,25.0,0.0,92346
3,4722,3831,1.028506e+09,3,Jaws (1975),1,0,0,0,0,...,0,0,0,0,0,0,M,25.0,0.0,92346
4,4722,1649,1.028508e+09,3,F/X (1986),1,0,0,0,0,...,0,0,0,1,0,0,M,25.0,0.0,92346
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134975,1973,3071,1.003618e+09,4,Hamlet (1948),0,0,0,0,0,...,0,0,0,0,0,0,F,1.0,19.0,14850
134976,4544,3416,9.747792e+08,3,Jimmy Hollywood (1994),0,0,0,0,1,...,0,0,0,0,0,0,M,25.0,16.0,94014
134977,2378,159,9.746208e+08,4,Little Odessa (1994),0,0,0,0,0,...,0,0,0,0,0,0,M,25.0,13.0,33316
134978,4955,1790,9.584218e+08,4,"Boys, The (1997)",0,0,0,0,0,...,0,0,0,0,0,0,M,25.0,6.0,19342


### Creating new csv files
To create the csv files of cleand data I split the merged dataset into three new dataset that contains the necessary columns. I have created the csv files `rangering.csv`, `film.csv` and `bruker.csv` that will be used in `model.ipynb`.

In [10]:
movie_df = merged.iloc[:, 4:23]
movie_df.insert(0, 'FilmID', merged.iloc[:, 1])
movie_df = movie_df.sort_values(by = 'FilmID').drop_duplicates(subset = 'FilmID').reset_index(drop = True)
movie_df

,FilmID,Tittel,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,0,Autumn in New York (2000),0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0
1,1,"Vie est belle, La (Life is Rosey) (1987)",0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
2,2,Defying Gravity (1997),0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3,3,Ruthless People (1986),0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
4,5,Defending Your Life (1991),0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3686,3948,Cat People (1982),0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
3687,3949,"Saltmen of Tibet, The (1997)",0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3688,3950,Bride of Re-Animator (1990),0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0
3689,3951,True Lies (1994),1,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0


In [11]:
rating_df = merged.iloc[:, :4]
rating_df = rating_df.sort_values(by = 'BrukerID').drop_duplicates().reset_index(drop = True)
rating_df

,BrukerID,FilmID,Tidsstempel,Rangering
0,0,791,959442983.0,2
1,0,2975,959443833.0,4
2,0,3407,959443373.0,4
3,0,189,959445005.0,4
4,0,773,959443944.0,2
...,...,...,...,...
899853,6040,10,976570594.0,5
899854,6040,2063,976300823.0,3
899855,6040,2398,976300671.0,3
899856,6040,2973,976300936.0,3


In [12]:
user_df = merged.iloc[:, 23:]
user_df.insert(0, 'BrukerID', merged.iloc[:, 0])
user_df = user_df.sort_values(by = 'BrukerID').drop_duplicates(subset='BrukerID').reset_index(drop = True)
user_df

,BrukerID,Kjonn,Alder,Jobb,Postkode
0,0,M,45.0,6.0,92103
1,1,M,50.0,16.0,55405
2,2,M,18.0,20.0,44089
3,3,M,25.0,1.0,33304
4,4,M,35.0,6.0,48105
...,...,...,...,...,...
6035,6036,M,45.0,0.0,61821
6036,6037,F,25.0,0.0,Unknown
6037,6038,M,25.0,16.0,33301
6038,6039,M,35.0,14.0,92075


In [13]:
rating_df.to_csv (r'/Users/kristinlokaoydna/UiB - Bioinformatikk/H20/INF161/Prosjekt/cleaned_data/rangering.csv', index = False, header=True)
movie_df.to_csv (r'/Users/kristinlokaoydna/UiB - Bioinformatikk/H20/INF161/Prosjekt/cleaned_data/film.csv', index = False, header=True)
user_df.to_csv (r'/Users/kristinlokaoydna/UiB - Bioinformatikk/H20/INF161/Prosjekt/cleaned_data/bruker.csv', index = False, header=True)